CSV解析器

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

chat=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser=CommaSeparatedListOutputParser()

format_instructions="您的响应应该是csv格式的逗号分隔值的列表，例如：'内容1','内容2','内容3'"
prompt=PromptTemplate(
    template="{format_instructions}\n请列出五个{subject}.",
    input_variables=["subject"],
    partial_variables={"format_instructions":format_instructions}
)

chain=prompt | chat | output_parser

In [3]:
format_instructions=output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [4]:
chain.invoke({"subject":"冰淇淋口味"})

["['巧克力'", "'草莓'", "'香草'", "'芒果'", "'蓝莓']"]

In [5]:
prompt.invoke({"subject":"冰淇淋口味"})

StringPromptValue(text="您的响应应该是csv格式的逗号分隔值的列表，例如：'内容1','内容2','内容3'\n请列出五个冰淇淋口味.")

日期解析器

In [6]:
from langchain.output_parsers import  DatetimeOutputParser
output_parser=DatetimeOutputParser()
template="""回答用户的问题：

{question}

{format_instructions}"""

format_instructions=output_parser.get_format_instructions()
format_instructions=''' 响应的格式用日期时间字符串："%Y-%m-%dT%H:%M:%S.%fZ" 。

示例：1898-05-31T06:59:40.248940Z, 1808-10-20T01:56:09.167633Z、0226-10-17T06:18:24.192024Z

仅返回此字符串，没有其他单词！'''

prompt=PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions":format_instructions}
)

In [7]:
format_instructions

' 响应的格式用日期时间字符串："%Y-%m-%dT%H:%M:%S.%fZ" 。\n\n示例：1898-05-31T06:59:40.248940Z, 1808-10-20T01:56:09.167633Z、0226-10-17T06:18:24.192024Z\n\n仅返回此字符串，没有其他单词！'

In [8]:
chain=prompt | chat | output_parser

In [13]:
output=chain.invoke({"question":"比特币是什么时候创立的"})
output

datetime.datetime(2009, 9, 7, 0, 0)

Enum parser 枚举解析器

In [14]:
from langchain.output_parsers import EnumOutputParser

In [15]:
from enum import Enum

class Colors(Enum):
    RED = "红色"
    BROWN = "棕色"
    BLACK = "黑色"
    WHITE = "白色"
    YELLOW = "黄色"

In [16]:
parser=EnumOutputParser(enum=Colors)

In [17]:
promptTemplate = PromptTemplate.from_template(
    """{person} 的皮肤主要是什么颜色？

{instructions}"""
)

# instructions=parser.get_format_instructions()
instructions = "响应的结果请选择以下选项之一：红色、棕色、黑色、白色、黄色。不要有其他的内容"
prompt = promptTemplate.partial(instructions=instructions)
chain = prompt | chat | parser

In [18]:
instructions=parser.get_format_instructions()
instructions

'Select one of the following options: 红色, 棕色, 黑色, 白色, 黄色'

In [19]:
chain.invoke({"person":"亚洲人"})

<Colors.YELLOW: '黄色'>